In [1]:
import poppusher
from icecream import ic
import pandas as pd
import re

In [ ]:
catalog: pd.DataFrame = poppusher.defs.load_asset_value(["be", "catalog_as_dataframe"])

geo_cat = catalog[
    catalog["source_format"].str.contains(
        "http://publications.europa.eu/resource/authority/file-type/GEOJSON"
    )
]


pop_stats = catalog[
    catalog["human_readable_name"].str.contains("Population by Statistical sector")
]
pop_stats = pop_stats[["node", "human_readable_name", "description"]]

for i, row in pop_stats.iterrows():
    print(row["node"])
    print(row["human_readable_name"])
    print(row["description"])
    print("~~~~~~~~~~~~~~~~~~~~~~~~")

# Statistical sectors 2023
# "node" : "https://statbel.fgov.be/node/4726",

In [ ]:
catalog: pd.DataFrame = poppusher.defs.load_asset_value(["gbr", "catalog"])

# catalog.head(1).T

current_table = catalog[catalog["partition_key"] == "oa/TS001"]

source_download_url = current_table["source_download_url"].values
source_archive_file_path = current_table["source_archive_file_path"].values[0]

ic(source_download_url)
ic(source_archive_file_path)

source_download_url = current_table["source_download_url"].to_numpy()
source_archive_file_path = current_table["source_archive_file_path"].to_numpy()[0]

ic(source_download_url)
ic(source_archive_file_path)

In [ ]:
geometry = poppusher.defs.load_asset_value(
    ["be", "individual_census_table"], partition_key="https://statbel.fgov.be/node/4726"
)
# Statistical sectors 2023
# geometry.head()
# ic(geometry.columns)
geometry.plot(legend=True)

In [ ]:
import requests

# Get description of the table
api_url = "https://www.nomisweb.co.uk/api/v01/dataset/nm_1_1.overview.json?select=DateMetadata,DatasetMetadata,Dimensions,DimensionMetadata"

api_url = "https://www.nomisweb.co.uk/api/v01/dataset/C2021TS001/overview.json?select=DatesetInfo"

api_url = "https://www.nomisweb.co.uk/api/v01/concept/C2021TS001.def.sdmx.json"


# DatasetInfo

In [ ]:
# Test aggregation of census2021-ts009-ltla

df = pd.read_csv("/Users/a.smith/Downloads/census2021-ts009/census2021-ts009-ltla.csv")
ic(df.shape)

# single_row = df.head(1)
# ic(single_row.T)

# Reproduce this example from NI:
# DerivedColumn(
#     hxltag="#population+children+age5_17",
#     filter_func=lambda df: df.query(f"{age_code} >= 5 and {age_code} < 18"),
#     output_column_name="children_5_17",
#     human_readable_name="Children aged 5 to 17",
# ),


def columns_for_5_17():
    regex_str = r"Sex: All persons; Age: Aged (?P<age>\d\d?) years?; measures: Value"
    regex = re.compile(regex_str)

    ic(list(range(5, 17)))

    columns_to_sum = []
    for col in df.columns:
        match = regex.search(col)
        if match and int(match.group("age")) in list(range(5, 18)):
            columns_to_sum.append(col)
    return columns_to_sum


#  [col for col in df.columns if ic(regex.search(col).group("age")) in ("0", "1")]
columns_to_sum = columns_for_5_17()
ic(columns_to_sum)

# filter_func = lambda row: sum([col in row for col in ("All ages", "All")])
func = lambda row: sum([row[col] for col in columns_to_sum])

# result = single_row.apply(filter_func, axis=1)
# ic(result)

new_table = df.assign(**{"new_col": df.apply(func, axis=1)}).filter(
    ["date", "geography", "geography code", "new_col"]
)
ic(new_table.shape)
ic(new_table.head())
ic(new_table.columns)
# for r in single_row.columns:
#     print(r)
# ic(new_table[['date', 'geography', 'geography code', 'new_col']])

In [ ]:
d = {"a": 1, "b": 2, "c": 3}

l = ["z", *d.keys()]
ic(l)

In [ ]:
# Understanding partials

from functools import partial


all_columns_to_sum = [
    ["a", "b", "c"],
    ["d", "e", "f", "g", "h", "i"],
    [
        "j",
        "k",
        "l",
        "m",
        "n",
        "o",
        "p",
        "q",
        "r",
        "s",
        "t",
        "u",
        "v",
        "w",
        "x",
        "y",
        "z",
    ],
]

all_funcs = []

for columns_to_sum in all_columns_to_sum:

    def func(char_list):
        # return sum([row[col] for col in columns_to_sum])
        # return partial(lambda x: "".join(x), char_list)
        return "".join(char_list)

    all_funcs.append(partial(func, char_list=columns_to_sum))


for my_func in all_funcs:
    print(my_func())
    # print(filter_func({"a": 1, "b": 2, "c": 3, "d": 4, "e": 5, "f": 6}))

# def filter_func(row):
#     return sum([row[col] for col in columns_to_sum])

In [ ]:
derived = poppusher.defs.load_asset_value(
    ["gbr", "census_tables"], partition_key="ltla/TS009"
)

ic(type(derived))
ic(derived.head(1).T)
# ic(derived.metadata)
# ic(derived.metrics.head(1).T)